# 🦜🔗 LangChain核心源代码解读（下）

# （二）解读源码，深度定制自己的智能体

## 下半场的闲聊

- 🌹 [LangChain 中的 Agent](https://python.langchain.com/docs/modules/agents/agent_types/)
- 🌹 [Chain 遗产](https://python.langchain.com/docs/modules/chains)

### 🌹 langchain中智能体相关类的清单

**Agent**
- AgentType
    - ZERO_SHOT_REACT_DESCRIPTION（**ReAct**的一般实现）
    - REACT_DOCSTORE（ReAct，支持RAG）
    - SELF_ASK_WITH_SEARCH（使用**search 工具**不断反思获得答案）
    - CONVERSATIONAL_REACT_DESCRIPTION（**ReAct**，支持对话）
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION（同上）
    - CHAT_CONVERSATIONAL_REACT_DESCRIPTION（同上）
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION（**ReAct**，为对话模型优化，支持多输入）
    - OPENAI_FUNCTIONS（支持**OpenAI Function Calling**）
    - OPENAI_MULTI_FUNCTIONS（支持**OpenAI Function Calling**，支持多函数调度）

**智能体执行器子组件：**

- AgentAction
    - AgentActionMessageLog
- AgentStep
- AgentFinish

**单动智能体：**
- **BaseSingleActionAgent**
    - RunnableAgent
    - LLMSingleActionAgent（__遗留__，__修改为__：create_***_agent）
    - XMLAgent（__遗留__，__修改为__：create_xml_agent）
    - Agent
        - ChatAgent（__遗留__，__修改为__：create_react_agent）
        - ConversationalAgent（__遗留__，__修改为__：create_react_agent）
        - ConversationalChatAgent（__遗留__，__修改为__：create_json_chat_agent）
        - StructuredChatAgent（__遗留__，__修改为__：create_structured_chat_agent）
        - ZeroShotAgent（__遗留__，__修改为__：create_react_agent）
        - ReActDocstoreAgent（__遗留__）
            - ReActTextWorldAgent（__遗留__）
        - SelfAskWithSearchAgent（__遗留__，__修改为__：create_self_ask_with_search）
    - OpenAIFunctionsAgent（__遗留__，__修改为__：create_openai_functions_agent）

**多动智能体：**
- BaseMultiActionAgent
    - RunnableMultiActionAgent
    - OpenAIMultiFunctionsAgent（__遗留__，__修改为__：create_openai_tools_agent）

**Assistant：**
- Runnable
    - RunnableSerializable
        - Chain
            - **AgentExecutor**
                - MRKLChain（__遗留__）
                - ReActChain（__遗留__）
                - SelfAskWithSearchChain（__遗留__）
        - **OpenAIAssistantRunnable**

**提示语模板：**
- Runnable
    - RunnableSerializable
        - BasePromptTemplate `[Dict, PromptValue]`
            - BaseChatPromptTemplate
                - ChatPromptTemplate
                    - **AgentScratchPadChatPromptTemplate**

**输出解析：**
- Runnable
    - RunnableSerializable
        - BaseOutputParser
            - **AgentOutputParser**
            - **MultiActionAgentOutputParser**

## 4、 openai风格智能体

### 🌹 从源码中搜寻 openai 的痕迹

- [partners/openai/langchain_openai/__init__.py](https://github.com/langchain-ai/langchain/blob/master/libs/partners/openai/langchain_openai/__init__.py)
- [core/langchain_core/utils/function_calling.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/utils/function_calling.py)
- [core/langchain_core/output_parsers/openai_tools.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_tools.py)
- [core/langchain_core/output_parsers/openai_functions.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_functions.py)

### 🌹 Function Calling 是怎么回事

### 🌹 create_openai_tool

## 5、 react风格智能体
### 🌹 create_react_agent

- 一个简单的智能体需求：与AI玩一个捉迷藏游戏
- react智能体的prompt如何工作
- 中间步骤的一步步产生过程
- action如何解析

## 6、在LCEL框架下重新《手撕AutoGPT》

### ✍️ 自定义智能体
### 🌹 智能体的 OutputParser

- 工具：定义一个简单工具
- 智能体：
    - Tools-Calling 智能体
    - ReAct 智能体
- 执行器：AgentExecutor

难点：
- 官方例子和内置智能体无法支持pydantic参数解析（智谱AI等推理能力较弱的模型可以使用）

### 🌹 AgentExcutor中的记忆体

## 7、Langgraph中的智能体

**一起来打开眼界**

# 结束了

## ❤️ 总结

- 我们一起阅读了langchain的源代码结构和部份细节
    - BaseLanguageModel / BaseChatModel / 
    - Runnable
    - LambdaRunnable
    - Chain
    - AgentExcutor
    - langgraph.prebuild
- 我们学习了如何自己动手集成大模型到 langchain 中
- 我们拆解了langchain的基石组件：Runnable
- 我们拆解了langchain的核心逻辑能力：LCEL
- 我们拆解了langchain的最新逻辑能力：langgraph
- 我们动手做了一些代码实践